In [1]:
# pip install torch

In [1]:
import pandas as pd
import numpy as np
import copy
from argparse import Namespace
from copy import deepcopy
import torch.nn as nn
from sklearn import preprocessing
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from folktables import ACSDataSource, ACSEmployment,ACSIncome
import matplotlib.pyplot as plt
import shutil
import torch.nn.functional as F
import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torchvision import models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from typing import Dict, List, OrderedDict, Tuple
import pickle
from collections import OrderedDict
from typing import OrderedDict
from typing import Dict, List, OrderedDict, Tuple, Union
from torch.utils.data import DataLoader, Subset, random_split
from collections import OrderedDict
import os
import random

from path import Path
from rich.console import Console
from rich.progress import track
from collections import OrderedDict
import os
import torch
import random
from collections import OrderedDict
from typing import List, OrderedDict, Tuple


In [2]:
import os
import shutil

old_model_path = "D:/Download/pythonProject/HiWi/pFedLA_Folktable/temp/my_model/"
client_models_path = "D:/Download/pythonProject/HiWi/pFedLA_Folktable/src/pkl_files/hn/"


for file in os.listdir(old_model_path):
    file_path = os.path.join(old_model_path, file)
    os.remove(file_path)
    print(f"Deleted: {file_path}")
    
for file in os.listdir(client_models_path):
    file_path = os.path.join(client_models_path, file)
    os.remove(file_path)
    print(f"Deleted: {file_path}")
        


print("All files removed successfully.")


Deleted: D:/Download/pythonProject/HiWi/pFedLA_Folktable/temp/my_model/clients_model.pt
Deleted: D:/Download/pythonProject/HiWi/pFedLA_Folktable/temp/my_model/epoch.pkl
Deleted: D:/Download/pythonProject/HiWi/pFedLA_Folktable/src/pkl_files/hn/0.pkl
Deleted: D:/Download/pythonProject/HiWi/pFedLA_Folktable/src/pkl_files/hn/1.pkl
Deleted: D:/Download/pythonProject/HiWi/pFedLA_Folktable/src/pkl_files/hn/2.pkl
Deleted: D:/Download/pythonProject/HiWi/pFedLA_Folktable/src/pkl_files/hn/3.pkl
All files removed successfully.


In [3]:

def fix_random_seed(seed: int) -> None:
    
#     print("fix_random_seed Chirag")
    
    torch.cuda.empty_cache()
    torch.random.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

    
def clone_parameters(
    src: Union[OrderedDict[str, torch.Tensor], torch.nn.Module]
) -> OrderedDict[str, torch.Tensor]:
    
#     print("clone_parameters Chirag")
    
    if isinstance(src, OrderedDict):
        return OrderedDict(
            {
                name: param.clone().detach().requires_grad_(param.requires_grad)
                for name, param in src.items()
            }
        )
    if isinstance(src, torch.nn.Module):
        return OrderedDict(
            {
                name: param.clone().detach().requires_grad_(param.requires_grad)
                for name, param in src.state_dict(keep_vars=True).items()
            }
        )

    
    
def get_dataloader(
    dataset: str,
    client_id: int,
    batch_size=10,
    valset_ratio=0.2,
    testset_ratio=0.1,
    only_dataset=False,
) -> Dict[str, Union[DataLoader, Subset]]:
    
    print("get_dataloader Chirag ")
    
    args_dict = json.load(open(DATASETS_DIR / "args.json", "r"))
    client_num_in_each_pickles = args_dict["client_num_in_each_pickles"]
    pickles_dir = DATASETS_DIR / dataset / "pickles"
    if os.path.isdir(pickles_dir) is False:
        raise RuntimeError("Please preprocess and create pickles first.")

    pickle_path = (
        pickles_dir / f"{math.floor(client_id / client_num_in_each_pickles)}.pkl"
    )
    with open(pickle_path, "rb") as f:
        subset = pickle.load(f)
    client_dataset = subset[client_id % client_num_in_each_pickles]
    val_samples_num = int(len(client_dataset) * valset_ratio)
    test_samples_num = int(len(client_dataset) * testset_ratio)
    train_samples_num = len(client_dataset) - val_samples_num - test_samples_num
    trainset, valset, testset = random_split(
        client_dataset, [train_samples_num, val_samples_num, test_samples_num]
    )
    if only_dataset:
        return {"train": trainset, "val": valset, "test": testset}
    trainloader = DataLoader(trainset, batch_size, shuffle=True)
    valloader = DataLoader(valset, batch_size)
    testloader = DataLoader(testset, batch_size)
    
    return {"train": trainloader, "val": valloader, "test": testloader}


In [4]:

class ClientBase:
    def __init__(
        self,
        backbone: torch.nn.Module,
        dataset: str,
        batch_size: int,
        valset_ratio: float,
        testset_ratio: float,
        local_epochs: int,
        local_lr: float,
        logger: Console,
        gpu: int,
    ):
        self.device = torch.device(
            "cuda" if gpu and torch.cuda.is_available() else "cpu"
        )
        self.client_id: int = None
        self.valset: DataLoader = None
        self.trainset: DataLoader = None
        self.testset: DataLoader = None
        
        
        # need to change
#         self.model: torch.nn.Module = init_model
        self.model: torch.nn.Module = deepcopy(backbone)
        
#         print("model client", self.model.parameters())
        
#         self.optimizer: torch.optim.Optimizer = torch.optim.SGD(
#             self.model.parameters(), lr=local_lr
#         )
            
        self.optimizer: torch.optim.Optimizer = torch.optim.Adam(
            self.model.parameters(), lr=local_lr
        )
          
        self.batch_size = batch_size
        self.valset_ratio = valset_ratio
        self.testset_ratio = testset_ratio
        self.local_epochs = local_epochs
        self.local_lr = local_lr
        self.criterion = torch.nn.BCELoss()
        self.logger = logger
        
        print("Base Client constructor")

#     @torch.no_grad()
#     def evaluate(self):
#         self.model.eval()
#         size = 0
#         loss = 0
#         correct = 0
#         for x, y in self.testset:
#             x, y = x.to(self.device), y.to(self.device)
            
#             # check this fun
            
#             logits = self.model(x)
                 
#             loss += self.criterion(logits, y)

#             pred = torch.softmax(logits, -1).argmax(-1)

#             correct += (pred == y).int().sum()

#             size += y.size(-1)
        
#         print("correct: %f\n" % (correct))
#         print("loss: %f\n" % (loss))
#         print("size: %f\n" % (size))
        
#         acc = correct / size * 100.0
        
#         print("Acc: %f\n" % (acc))
        
#         loss = loss / len(self.testset)
#         return loss, acc
    
    
    @torch.no_grad()
    def evaluate(self):
        self.model.eval()
        size = 0
        loss = 0
        correct = 0
        total = 0
        for inputs, targets in self.testset:
            
            inputs, targets = inputs.to(self.device), targets.to(self.device)
            
            outputs = self.model(inputs)
            loss += self.criterion(outputs, targets)
            predicted = outputs > 0.6
            
            
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            
#             acc += (y_pred.round() == y).float().mean()
        
                
#         print("Correct: ",correct,"Total: ", total)
        loss = loss / len(self.testset)
        acc = correct / total
 
#         print("loss: %f\n" % (loss))
        print(f"Testing Accuracy: {acc:.2%}")
        
        return loss, acc
    

    def train(self):
        print("Base Client train called")
        pass

    def _train(self):
        print("Base Client __train called")
        pass

    def get_client_local_dataset(self):
        
#         datasets = get_dataloader(
#             self.dataset,
#             self.client_id,
#             self.batch_size,
#             self.valset_ratio,
#             self.testset_ratio,
#         )

        print("self.client_id:: ",self.client_id)
                                
#         with open("..\\train_1_client.pkl", "rb") as f:
#             self.trainset = pickle.load(f)
            
        with open("..\\train_all_client.pkl", "rb") as f:
            self.trainset = pickle.load(f)
            
        with open("..\\valid_all_client.pkl", "rb") as f:
            self.valset = pickle.load(f)

#         with open("..\\valid_1_client.pkl", "rb") as f:
#             self.valset  = pickle.load(f)
        
        with open("..\\test_client.pkl", "rb") as f:
            self.testset  = pickle.load(f)
        
        self.trainset = self.trainset[self.client_id]
        self.valset = self.valset[self.client_id]       
#         self.testset = self.testset

                
#         print("self.trainset",self.trainset)
        
#         self.trainset = datasets["train"]
#         self.valset = datasets["val"]        
#         self.testset = datasets["test"]

    def _log_while_training(self, evaluate=True, verbose=False):
        def _log_and_train(*args, **kwargs):
            loss_before = 0
            loss_after = 0
            acc_before = 0
            acc_after = 0
            if evaluate:
                loss_before, acc_before = self.evaluate()

            res = self._train(*args, **kwargs)

            if evaluate:
                loss_after, acc_after = self.evaluate()
            
            
#             if verbose:
#                 self.logger.log(
#                     "client [{}]   [bold red]loss: {:.4f} -> {:.4f}    [bold blue]accuracy: {:.2f}% -> {:.2f}%".format(
#                         self.client_id, loss_before, loss_after, acc_before, acc_after
#                     )
#                 )

            eval_stats = {
                "loss_before": loss_before,
                "loss_after": loss_after,
                "acc_before": acc_before,
                "acc_after": acc_after,
            }
            
            return res, eval_stats

        return _log_and_train

    def set_parameters(self, model_params: OrderedDict):

        self.model.load_state_dict(model_params, strict=True)


In [5]:


class pFedLAClient(ClientBase):
    def __init__(
        self,
        backbone: torch.nn.Module,        
        dataset: str,
        batch_size: int,
        valset_ratio: float,
        testset_ratio: float,
        local_epochs: int,
        local_lr: float,
        logger: Console,
        gpu: int,
    ):
        super(pFedLAClient, self).__init__(
            backbone,
            dataset,
            batch_size,
            valset_ratio,
            testset_ratio,
            local_epochs,
            logger,
            local_lr,
            gpu,
        )
        print("PFedLa Client constructor")
        self.all_client_info=[]
    
    def train(
        self,
        client_id: int,
        model_params: OrderedDict[str, torch.Tensor],
        verbose=True,
    ): 
#         print("pFedL Client train called")
        self.client_id = client_id
        
#         model_params= DeepNet()

        self.set_parameters(model_params)                
        self.get_client_local_dataset()
        self.model.to(self.device)
        
        res, stats = self._log_while_training(evaluate=True, verbose=verbose)()
        self.model.cpu()
        
        self.all_client_info.append(res)
        return res, stats
        
    def _train(self):
#         print("pFedL Client __train called")
        self.model.train()
        frz_model_params = clone_parameters(self.model)

        for _ in range(self.local_epochs):
            for x, y in self.trainset:
                x, y = x.to(self.device), y.to(self.device)

                logits = self.model(x)

                loss = self.criterion(logits, y)

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

        delta = OrderedDict(
            {
                k: p1 - p0 
                for (k, p1), p0 in zip(
                    self.model.state_dict(keep_vars=True).items(),
                    frz_model_params.values(),
                )
            }
        )
        
        return delta
    
    def test(
        self, client_id: int, model_params: OrderedDict[str, torch.Tensor],
    ):
        self.client_id = client_id
        self.set_parameters(model_params)
        self.get_client_local_dataset()
        self.model.to(self.device)
        loss, acc = self.evaluate()
        dummy_diff = OrderedDict(
            {
                name: torch.zeros_like(param)
                for name, param in self.model.state_dict().items()
            }
        )
        self.model.cpu()
        stats = {"loss": loss, "acc": acc}
        return dummy_diff, stats 

In [6]:
class DeepNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 14 : input shape
        self.layer1 = nn.Linear(14, 512)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)
        self.layer2 = nn.Linear(512, 256)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(256, 60)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(60, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.dropout1(x)
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x

In [7]:

class Linear(nn.Module):
    def __init__(self, in_features, out_features) -> None:
        super().__init__()

        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.empty(out_features, in_features))
        self.bias = nn.Parameter(torch.empty(out_features))

        nn.init.uniform_(self.weight)
        nn.init.constant_(self.bias, 0.0)

    def forward(self, x):
        return F.linear(x, self.weight, self.bias)

    

class HyperNetwork(nn.Module):
    def __init__(
        self,
        embedding_dim: int,
        hidden_dim: int,
        backbone: nn.Module,
        client_num: int,
        K: int,
        gpu=True,
    ):
    
        super(HyperNetwork, self).__init__()
        self.device = torch.device(
                "cuda" if gpu and torch.cuda.is_available() else "cpu"
            )
        
        self.K = K
        self.client_num = client_num
        self.embedding = nn.Embedding(client_num, embedding_dim, device=self.device)
        self.blocks_name = set(n.split(".")[0] for n, _ in backbone.named_parameters())
        self.cache_dir =  "pkl_files/hn/"  # put dir here
        
        if os.listdir(self.cache_dir) != client_num:
            
            for client_id in range(client_num):
#                 with open(self.cache_dir / f"{client_id}.pkl", "wb") as f:
                with open(os.path.join(self.cache_dir, f"{client_id}.pkl"), "wb") as f:

                    pickle.dump(
                        {
                            "mlp": nn.Sequential(
                                nn.Linear(embedding_dim, hidden_dim),
                                nn.ReLU(),
                                nn.Linear(hidden_dim, hidden_dim),
                                nn.ReLU(),
                            ),
                            
                            "fc": {
                                name: Linear(hidden_dim, client_num)
                                for name in self.blocks_name
                            },
                        },
                        f,
                    )

        # for tracking the current client's hn parameters
        self.current_client_id: int = None
        self.mlp: nn.Sequential = None
        self.fc_layers: Dict[str, Linear] = {}
        self.retain_blocks: List[str] = []
            
#         print("HypterNetwork")

    def mlp_parameters(self) -> List[nn.Parameter]:
            return list(filter(lambda p: p.requires_grad, self.mlp.parameters()))           
        
    def fc_layer_parameters(self) -> List[nn.Parameter]:
        params_list = []
        for block, fc in self.fc_layers.items():
            if block not in self.retain_blocks:
                params_list += list(filter(lambda p: p.requires_grad, fc.parameters()))                
        return params_list

    def emd_parameters(self) -> List[nn.Parameter]:
        return list(self.embedding.parameters())        
    
    def forward(self, client_id: int) -> Tuple[Dict[str, torch.Tensor], List[str]]:
        self.current_client_id = client_id

        self.retain_blocks = []
        
        emd = self.embedding(
            torch.tensor(client_id, dtype=torch.long, device=self.device)
        )
        self.load_hn()

        feature = self.mlp(emd)

        alpha = {
            block: F.relu(self.fc_layers[block](feature)) for block in self.blocks_name
        }

        

        default_weight = torch.tensor(
            [i == client_id for i in range(self.client_num)],
            dtype=torch.float,
            device=self.device,
        )

        if self.K > 0:  # HeurpFedLA
            
            blocks_name = []
            self_weights = []
            
            with torch.no_grad():
                for name, weight in alpha.items():

                    blocks_name.append(name)
                    self_weights.append(weight[client_id])

                # not in the Loop
                _, topk_weights_idx = torch.topk(torch.tensor(self_weights), self.K)
                
            for i in topk_weights_idx:
                alpha[blocks_name[i]] = default_weight
                self.retain_blocks.append(blocks_name[i])
#             print("  alpha: ",alpha,"\n")
        return alpha, self.retain_blocks
        
    
    def save_hn(self):
            for block, param in self.fc_layers.items():
                self.fc_layers[block] = param.cpu()
                
            file_path = os.path.join(self.cache_dir, f"{self.current_client_id}.pkl")
            with open(file_path, "wb") as f:
#             with open(self.cache_dir / f"{self.current_client_id}.pkl", "wb") as f:
                pickle.dump(
                    {"mlp": self.mlp.cpu(), "fc": self.fc_layers}, f,
                )
            self.mlp = None
            self.fc_layers = {}
            self.current_client_id = None

    def load_hn(self) -> Tuple[nn.Sequential, OrderedDict[str, Linear]]:
        with open(os.path.join(self.cache_dir, f"{self.current_client_id}.pkl"), "rb") as f:
        
#         with open(self.cache_dir / f"{self.current_client_id}.pkl", "rb") as f:
            parameters = pickle.load(f)
        self.mlp = parameters["mlp"].to(self.device)
        for block, param in parameters["fc"].items():
            self.fc_layers[block] = param.to(self.device)

    def clean_models(self):
        if os.path.isdir(self.cache_dir):
            os.system(f"rm -rf {self.cache_dir}")
            


In [8]:
class ServerBase:
    def __init__(self, args: Namespace, algo: str):
        self.algo = algo
        self.args = args
        
        # default log file format
        self.log_name = "{}_{}_{}_{}.html".format(
            self.algo,
            self.args.dataset,
            self.args.global_epochs,
            self.args.local_epochs,
        )

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        fix_random_seed(self.args.seed)
        
#         self.backbone= (DeepNet)   
        
        self.backbone = (
            CNNWithBatchNorm
            if self.args.dataset in ["cifar10", "cifar100"]
            else DeepNet
        )
        
        print("self.backbone",self.backbone)
        
        self.logger = Console(record=True, log_path=False, log_time=False,)
        
        self.client_id_indices, self.client_num_in_total = [0,1,2,3], 4                
        
        self.temp_dir ="D:/Download/pythonProject/HiWi/pFedLA_Folktable/temp/my_model/"

#         name_of_model = '../../WW_WM_BW.pth'
        #  self.temp_dir = TEMP_DIR / self.algo
        if not os.path.isdir(self.temp_dir):
            os.makedirs(self.temp_dir)
            
#         _dummy_model = self.backbone(self.args.dataset)
        _dummy_model = self.backbone()
#         _dummy_model = DeepNet()
    
#         print("_dummy_model:: base",_dummy_model)
        
        passed_epoch = 0
        self.global_params_dict: OrderedDict[str : torch.Tensor] = None
        
        if os.listdir(self.temp_dir) != []:
            if os.path.exists(os.path.join(self.temp_dir, "global_model.pt")):
                
                self.global_params_dict = torch.load(self.temp_dir / "global_model.pt")
                self.logger.log("Find existed global model...")

            if os.path.exists(os.path.join(self.temp_dir, "epoch.pkl")):  
                with open(os.path.join(self.temp_dir, "epoch.pkl"), "rb") as f:
#               
                    passed_epoch = pickle.load(f)
                self.logger.log(f"Have run {passed_epoch} epochs already.",)
        else:
            self.global_params_dict = OrderedDict(_dummy_model.state_dict())
            
#         self.global_params_dict = OrderedDict(_dummy_model.state_dict())
#         print("self.global_params_dict: ",self.global_params_dict)

        self.global_epochs = self.args.global_epochs - passed_epoch
#         self.global_epochs = self.global_epochs - passed_epoch
    
        self.logger.log("Backbone:", _dummy_model)

        self.trainer: ClientBase = None
        self.all_clients_stats = {i: {} for i in self.client_id_indices}
           
            
        print("Base Server Called !")
        
    def train(self):

        print("In Base server  train\n")
        
        self.logger.log("=" * 30, "TRAINING", "=" * 30, style="bold green")
        progress_bar = (
            track(
                range(self.global_epochs),
                "[bold green]Training...",
                console=self.logger,
            )
            if not self.args.log
            else tqdm(range(self.global_epochs), "Training...")
        )
        for E in progress_bar:

            if E % self.args.verbose_gap == 0:
                self.logger.log("=" * 30, f"ROUND: {E}", "=" * 30)

            selected_clients = random.sample(
                self.client_id_indices, self.args.client_num_per_round
            )
            
            updated_params_cache = []
            weights_cache = []

            for client_id in selected_clients:
                
                client_local_params = clone_parameters(self.global_params_dict)
                                
                (updated_params, weight), stats = self.trainer.train(
                    client_id=client_id,
                    model_params=client_local_params,
                    verbose=(E % self.args.verbose_gap) == 0,
                )

                updated_params_cache.append(updated_params)
                weights_cache.append(weight)
                self.all_clients_stats[client_id][f"ROUND: {E}"] = (
                    f"{stats['loss_before']:.4f} -> {stats['loss_after']:.4f}",
                )

            self.aggregate_parameters(updated_params_cache, weights_cache)

            if E % self.args.save_period == 0:
                torch.save(
                    self.global_params_dict, self.temp_dir / "global_model.pt",
                )
                with open(self.temp_dir / "epoch.pkl", "wb") as f:
                    pickle.dump(E, f)
        self.logger.log(self.all_clients_stats)
    
    @torch.no_grad()
    def aggregate_parameters(self, updated_params_cache, weights_cache):
        weight_sum = sum(weights_cache)
        weights = torch.tensor(weights_cache, device=self.device) / weight_sum

        aggregated_params = []

        for params in zip(*updated_params_cache):
            aggregated_params.append(
                torch.sum(weights * torch.stack(params, dim=-1), dim=-1)
            )

        self.global_params_dict = OrderedDict(
            zip(self.global_params_dict.keys(), aggregated_params)
        )

                
    def test(self) -> None:
        self.logger.log("=" * 30, "TESTING", "=" * 30, style="bold blue")
        all_loss = []
        all_acc = []
        for client_id in track(
            self.client_id_indices,
            "[bold blue]Testing...",
            console=self.logger,
            disable=self.args.log,
        ):
            client_local_params = clone_parameters(self.global_params_dict)
            stats = self.trainer.test(
                client_id=client_id, model_params=client_local_params,
            )

            self.logger.log(
                f"client [{client_id}]  [red]loss: {stats['loss']:.4f}    [magenta]accuracy: {stats['acc']:.2f}%"
            )
            all_loss.append(stats["loss"])
            all_acc.append(stats["acc"])

        self.logger.log("=" * 20, "RESULTS", "=" * 20, style="bold green")
        self.logger.log(
            "loss: {:.4f}    accuracy: {:.2f}%".format(
                sum(all_loss) / len(all_loss), sum(all_acc) / len(all_acc),
            )
        )        
    
    def run(self):
        print("Base server Run called")
        self.logger.log("Arguments:", dict(self.args._get_kwargs()))
        self.train()
        self.test()
        if self.args.log:
            if not os.path.isdir(LOG_DIR):
                os.mkdir(LOG_DIR)
            self.logger.save_html(LOG_DIR / self.log_name)

        # delete all temporary files
#         if os.listdir(self.temp_dir) != []:
#             os.system(f"rm -rf {self.temp_dir}")


In [9]:
args1 = Namespace(
    k=2,
    global_epochs=5,
    local_epochs=2,
    local_lr=0.001,
    hn_lr=0.002,
    verbose_gap=20,
    embedding_dim=200,
    hidden_dim=200,
    dataset="no",
    batch_size=1000,
    valset_ratio=0.0,
    testset_ratio=0.3,
    gpu=True,
    log=0,
    seed=5,
    client_num_per_round=2,
    save_period=10,
)


In [10]:

class pFedLAServer(ServerBase):
    def __init__(self):
        super(pFedLAServer, self).__init__(args1, "pFedLA")
        
        self.log_name = "{}_{}_{}_{}_{}.html".format(
            self.algo,
            self.args.dataset,
            self.args.global_epochs,
            self.args.local_epochs,
            self.args.k,
        )
        
        self.tracker_temp=0
        self.tracker_diff=[]
        
        if self.global_params_dict is not None:
            del self.global_params_dict  # pFedLA don't have global model

        if os.listdir(self.temp_dir) != []:             
            if os.path.exists(os.path.join(self.temp_dir, "clients_model.pt")):
                self.client_model_params_list = torch.load(
                    os.path.join(self.temp_dir, "clients_model.pt")
                )
                self.logger.log("Find existed clients model...")
        else:
            self.logger.log("Initializing clients model...")
            
            self.client_model_params_list = [
#                 list(self.backbone(self.args.dataset).state_dict().values())
                list(self.backbone().state_dict().values())
                for _ in range(self.client_num_in_total)
            ]
            
#         print("client_model_params_list:: Pserver ",self.client_model_params_list)
            
        _dummy_model = self.backbone()
        
#         print("_dummy_model Pserver :: ",_dummy_model)

#         _dummy_model = DeepNet()
      
#         _dummy_model = self.backbone(self.args.dataset)
        
    
        self.hypernet = HyperNetwork(
            embedding_dim=self.args.embedding_dim,
            client_num=self.client_num_in_total,
            hidden_dim=self.args.hidden_dim,
            backbone=_dummy_model,
            K=self.args.k,
            gpu=self.args.gpu,
        )

#         self.trainer = pFedLAClient(
#             backbone=_dummy_model,
#             dataset=self.args.dataset,
#             batch_size=self.args.batch_size,
#             valset_ratio=self.args.valset_ratio,
#             testset_ratio=self.args.testset_ratio,
#             local_epochs=self.args.local_epochs,
#             local_lr=self.args.local_lr,
#             logger=self.logger,
#             gpu=self.args.gpu,
#         )
        

        self.trainer = pFedLAClient(
            backbone=_dummy_model,
            dataset=self.args.dataset,
            batch_size=self.args.batch_size,
            valset_ratio=self.args.testset_ratio,
            testset_ratio=self.args.testset_ratio,
            local_epochs=self.args.local_epochs,
            local_lr=self.args.local_lr,
            logger=0,
            gpu=self.args.gpu,
        )
        
        self.all_params_name = [name for name in _dummy_model.state_dict().keys()]     
        
#         self.all_params_name=_dummy_model
        
        self.trainable_params_name = [
                name
                for name, param in _dummy_model.state_dict(keep_vars=True).items()
                if param.requires_grad
            ]
        
        
    def train(self) -> None:
#         print("PfedL server Train called")
        self.logger.log("=" * 30, "TRAINING", "=" * 30, style="bold green")
        progress_bar = (
            track(
                range(self.global_epochs),
                "[bold green]Training...",
                console=self.logger,
            )
            if not self.args.log
            else tqdm(range(self.global_epochs), "Training...")
        )        
        
        for E in progress_bar:

            if E % self.args.verbose_gap == 0:
                self.logger.log("=" * 30, f"ROUND: {E}", "=" * 30)

            selected_clients = random.sample(
                self.client_id_indices, self.args.client_num_per_round
            )
            
            print("self.global_epochs:: E, progress_bar ", E," :: ",progress_bar)
            
            for client_id in selected_clients:
                (
                    client_local_params,
                    retain_blocks,
                ) = self.generate_client_model_parameters(client_id)
                
#                 print("client_local_params:: \n",client_local_params)
                
                diff, stats = self.trainer.train(
                    client_id=client_id,
                    model_params=client_local_params,
                    verbose=(E % self.args.verbose_gap) == 1,
                )
                
                self.all_clients_stats[client_id][f"ROUND: {E}"] = (
                    f"retain {retain_blocks}, {stats['loss_before']:.4f} -> {stats['loss_after']:.4f}",
                )
                
                self.tracker_diff.append(diff)

                self.update_hypernetwork(client_id, diff, retain_blocks)

                self.update_client_model_parameters(client_id, diff)

            
            if E % self.args.save_period == 0:
                torch.save(self.client_model_params_list, os.path.join(self.temp_dir, "clients_model.pt"))
                with open(os.path.join(self.temp_dir, "epoch.pkl"), "wb") as f:
                    pickle.dump(E, f)
                    
        self.logger.log(self.all_clients_stats)
    
    
    def test(self) -> None:
        
        print("----------testing------")
        self.logger.log("=" * 30, "TESTING", "=" * 30, style="bold blue")
        all_loss = []
        all_acc = []
        for client_id in track(
            self.client_id_indices,
            "[bold blue]Testing...",
            console=self.logger,
            disable=self.args.log,
        ):
            client_local_params, retain_blocks = self.generate_client_model_parameters(
                client_id
            )
            dummy_diff, stats = self.trainer.test(
                client_id=client_id, model_params=client_local_params,
            )

            # NOTE: make sure that all client model params are on CPU, not CUDA
            # or self.generate_...() would raise the error of stacking tensors on different devices
            self.update_client_model_parameters(client_id, dummy_diff)
            self.logger.log(
                f"client [{client_id}] retain {retain_blocks}, [red]loss: {stats['loss']:.4f}    [magenta]accuracy: {stats['acc']:.2f}%"
            )
            all_loss.append(stats["loss"])
            all_acc.append(stats["acc"])

        self.logger.log("=" * 20, "RESULTS", "=" * 20, style="bold green")
        self.logger.log(
            "loss: {:.4f}    accuracy: {:.2f}%".format(
                sum(all_loss) / len(all_loss), sum(all_acc) / len(all_acc),
            )
        )

    
    @torch.no_grad()
    def update_client_model_parameters(
        self, client_id: int, delta: OrderedDict[str, torch.Tensor],
    ) -> None:
        updated_params = []
        for param, diff in zip(
            self.client_model_params_list[client_id], delta.values()
        ):
            updated_params.append((param + diff).detach().cpu())
        self.client_model_params_list[client_id] = updated_params

    
    def generate_client_model_parameters(
            self, client_id: int
        ) -> Tuple[OrderedDict[str, torch.Tensor], List[str]]:
            layer_params_dict = dict(
                zip(self.all_params_name, list(zip(*self.client_model_params_list)))
            )
            alpha, retain_blocks = self.hypernet(client_id)
            aggregated_parameters = {}
            default_weight = torch.tensor(
                [i == client_id for i in range(self.client_num_in_total)],
                dtype=torch.float,
                device=self.device,
            )
            for name in self.all_params_name:
                if name in self.trainable_params_name:
                    a = alpha[name.split(".")[0]]
                else:
                    a = default_weight
                if a.sum() == 0:
                    self.logger.log(self.all_clients_stats)
                    raise RuntimeError(
                        f"client [{client_id}]'s {name.split('.')[0]} alpha is a all 0 vector"
                    )
                    
                self.tracker_temp=a
                
                aggregated_parameters[name] = torch.sum(
                    a
                    / a.sum()
                    * torch.stack(layer_params_dict[name], dim=-1).to(self.device),
                    dim=-1,
                )

            self.client_model_params_list[client_id] = list(aggregated_parameters.values())
            
            return aggregated_parameters, retain_blocks
    
    def update_hypernetwork(
        self,
        client_id: int,
        diff: OrderedDict[str, torch.Tensor],
        retain_blocks: List[str] = [],
    ) -> None:
        # calculate gradients

        hn_grads = torch.autograd.grad(
            outputs=list(
                filter(
                    lambda param: param.requires_grad,
                    self.client_model_params_list[client_id],
                )
            ),
            inputs=self.hypernet.mlp_parameters()
            + self.hypernet.fc_layer_parameters()
            + self.hypernet.emd_parameters(),

            grad_outputs=list(
                map(
                    lambda tup: tup[1],
                    filter(
                        lambda tup: tup[1].requires_grad
                        and tup[0].split(".")[0] not in retain_blocks,
                        diff.items(),
                    ),
                )
            ),
            allow_unused=True,
        )
        
        mlp_grads = hn_grads[: len(self.hypernet.mlp_parameters())]
        fc_grads = hn_grads[
            len(self.hypernet.mlp_parameters()) : len(
                self.hypernet.mlp_parameters() + self.hypernet.fc_layer_parameters()
            )
        ]
        emd_grads = hn_grads[
            len(self.hypernet.mlp_parameters() + self.hypernet.fc_layer_parameters()) :
        ]

        for param, grad in zip(self.hypernet.fc_layer_parameters(), fc_grads):
            if grad is not None:
                param.data -= self.args.hn_lr * grad

        for param, grad in zip(self.hypernet.mlp_parameters(), mlp_grads):
            param.data -= self.args.hn_lr * grad

        for param, grad in zip(self.hypernet.emd_parameters(), emd_grads):
            param.data -= self.args.hn_lr * grad

        self.hypernet.save_hn()
        
        
    def run(self):
        print("PfedL server RUN called")        
        super().run()
        # clean out all HNs 
        self.hypernet.clean_models()


In [ ]:
# FedAvg and FairFed

In [11]:
server = pFedLAServer()

self.backbone <class '__main__.DeepNet'>


Backbone: DeepNet(                                                                                                 
  (layer1): Linear(in_features=14, out_features=512, bias=True)                                                    
  (act1): ReLU()                                                                                                   
  (dropout1): Dropout(p=0.5, inplace=False)                                                                        
  (layer2): Linear(in_features=512, out_features=256, bias=True)                                                   
  (act2): ReLU()                                                                                                   
  (layer3): Linear(in_features=256, out_features=60, bias=True)                                                    
  (act3): ReLU()                                                                                                   
  (output): Linear(in_features=60, out_features=1, bias=True)                                                      
  (sigmoid): Sigmoid()                                                                                             
)

Base Server Called !


Initializing clients model...

Base Client constructor
PFedLa Client constructor


In [ ]:
server.run()

In [14]:
retain_blocks=['layer2.weight','layer3.weight']

In [15]:
server.tracker_diff

[OrderedDict([('layer1.weight',
               tensor([[0., 0., 0.,  ..., 0., 0., 0.],
                       [0., 0., 0.,  ..., 0., 0., 0.],
                       [0., 0., 0.,  ..., 0., 0., 0.],
                       ...,
                       [0., 0., 0.,  ..., 0., 0., 0.],
                       [0., 0., 0.,  ..., 0., 0., 0.],
                       [0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<SubBackward0>)),
              ('layer1.bias',
               tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [16]:
map( lambda tup: tup[1],
                    filter(
                        lambda tup: tup[1].requires_grad and tup[0].split(".")[0] not in retain_blocks,
                        server.tracker_diff[0].items(),
                    ),
                )
            

In [17]:
params_list = []
for block, fc in server.hypernet.fc_layers.items():
    if block not in retain_blocks:
        params_list += list(filter(lambda p: p.requires_grad, fc.parameters()))

In [18]:
len(params_list)

8

In [35]:
server.hypernet.fc_layers.items()

dict_items([('layer3', Linear()), ('layer1', Linear()), ('output', Linear()), ('layer2', Linear())])

In [28]:
server.tracker_diff[0].items()

odict_items([('layer1.weight', tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<SubBackward0>)), ('layer1.bias', tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [20]:
p=list(
                map(
                    lambda tup: tup[1],
                    filter(
                        lambda tup: tup[1].requires_grad
                        and tup[0].split(".")[0] not in retain_blocks,
                        server.tracker_diff[0].items(),
                    ),
                )
            )

In [25]:
len(server.hypernet.mlp_parameters()),len(server.hypernet.fc_layer_parameters())

(4, 4)

In [26]:
p[:4]

[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<SubBackward0>),
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [50]:
server.all_params_name

['layer1.weight',
 'layer1.bias',
 'layer2.weight',
 'layer2.bias',
 'layer3.weight',
 'layer3.bias',
 'output.weight',
 'output.bias']

In [16]:
layer_params_dict = dict(zip(server.all_params_name, list(zip(*server.client_model_params_list))))

In [18]:
layer_params_dict

{'layer1.weight': (tensor([[ 0.1397, -0.2572, -0.0890,  ...,  0.0035, -0.0211,  0.0634],
          [-0.2041,  0.2103, -0.0196,  ..., -0.0148,  0.0740, -0.0785],
          [ 0.1840, -0.1573,  0.1232,  ...,  0.1452,  0.1784,  0.1457],
          ...,
          [ 0.1272,  0.1901, -0.0585,  ...,  0.0350,  0.1233, -0.1788],
          [ 0.0828, -0.1304, -0.0977,  ...,  0.0419,  0.0551,  0.1226],
          [ 0.0256,  0.1011, -0.0308,  ..., -0.0581, -0.1986, -0.0934]]),
  tensor([[ 0.0968, -0.0463, -0.0150,  ..., -0.0435, -0.0823,  0.1113],
          [ 0.0008,  0.0966,  0.0209,  ..., -0.0283, -0.0732, -0.0203],
          [ 0.1599, -0.0813,  0.1718,  ...,  0.0765,  0.0846,  0.0169],
          ...,
          [ 0.1794,  0.0361,  0.0829,  ...,  0.0166,  0.0558,  0.0058],
          [ 0.0542, -0.1225,  0.0591,  ..., -0.0765,  0.1391, -0.0342],
          [-0.0202,  0.0192,  0.0778,  ...,  0.0370, -0.1329, -0.0143]]),
  tensor([[ 0.1077, -0.1120, -0.0364,  ..., -0.0214, -0.0503,  0.0946],
          [-0

In [136]:
name_of_model="../temp/my_model/clients_model.pt"
model = DeepNet()
model.state_dict()

OrderedDict([('layer1.weight',
              tensor([[ 0.2483, -0.2280, -0.1916,  ...,  0.2507, -0.2441, -0.0449],
                      [-0.0039,  0.1995, -0.1238,  ...,  0.0322,  0.0309,  0.2374],
                      [ 0.2649, -0.1241,  0.0101,  ..., -0.2495,  0.1897,  0.1550],
                      ...,
                      [ 0.0244, -0.0109,  0.1211,  ..., -0.2113,  0.1134,  0.1280],
                      [ 0.0406, -0.1969,  0.0639,  ..., -0.1711, -0.2250,  0.1443],
                      [ 0.2634,  0.2226,  0.1362,  ...,  0.2453, -0.1734, -0.2258]])),
             ('layer1.bias',
              tensor([-2.5814e-01, -2.0460e-01, -8.3908e-02, -1.9310e-01, -1.0837e-01,
                       2.3699e-01, -1.0015e-01,  1.9833e-01,  2.6474e-02, -1.6649e-01,
                      -3.8439e-02,  3.2512e-02,  1.0392e-01, -1.9034e-01, -1.9774e-01,
                       2.7782e-02,  7.8452e-02, -1.3228e-01,  1.8627e-01, -8.3166e-03,
                       8.1070e-02, -3.7957e-02, -3.1808e-0

In [144]:
checkpoint = torch.load(name_of_model)[0]

model_state_dict = model.state_dict()

for name, param in model.named_parameters():
    if name in server.all_params_name:        
        param.data = checkpoint[server.all_params_name.index(name)]

model.state_dict()

OrderedDict([('layer1.weight',
              tensor([[ 0.1397, -0.2572, -0.0890,  ...,  0.0035, -0.0211,  0.0634],
                      [-0.2041,  0.2103, -0.0196,  ..., -0.0148,  0.0740, -0.0785],
                      [ 0.1840, -0.1573,  0.1232,  ...,  0.1452,  0.1784,  0.1457],
                      ...,
                      [ 0.1272,  0.1901, -0.0585,  ...,  0.0350,  0.1233, -0.1788],
                      [ 0.0828, -0.1304, -0.0977,  ...,  0.0419,  0.0551,  0.1226],
                      [ 0.0256,  0.1011, -0.0308,  ..., -0.0581, -0.1986, -0.0934]])),
             ('layer1.bias',
              tensor([-0.0993, -0.1852, -0.0704, -0.1630,  0.1803, -0.1084,  0.0895,  0.1346,
                       0.0764, -0.2357,  0.1187,  0.0415, -0.1458, -0.1661,  0.2115, -0.0123,
                       0.0293,  0.0544,  0.0375,  0.2155, -0.0440, -0.0033, -0.0887, -0.1519,
                       0.0771, -0.1158,  0.2551, -0.0163,  0.1748,  0.2399, -0.0200, -0.0989,
                       0.0458,

In [141]:
model.state_dict().values()

odict_values([tensor([[ 0.1397, -0.2572, -0.0890,  ...,  0.0035, -0.0211,  0.0634],
        [-0.2041,  0.2103, -0.0196,  ..., -0.0148,  0.0740, -0.0785],
        [ 0.1840, -0.1573,  0.1232,  ...,  0.1452,  0.1784,  0.1457],
        ...,
        [ 0.1272,  0.1901, -0.0585,  ...,  0.0350,  0.1233, -0.1788],
        [ 0.0828, -0.1304, -0.0977,  ...,  0.0419,  0.0551,  0.1226],
        [ 0.0256,  0.1011, -0.0308,  ..., -0.0581, -0.1986, -0.0934]]), tensor([-0.0993, -0.1852, -0.0704, -0.1630,  0.1803, -0.1084,  0.0895,  0.1346,
         0.0764, -0.2357,  0.1187,  0.0415, -0.1458, -0.1661,  0.2115, -0.0123,
         0.0293,  0.0544,  0.0375,  0.2155, -0.0440, -0.0033, -0.0887, -0.1519,
         0.0771, -0.1158,  0.2551, -0.0163,  0.1748,  0.2399, -0.0200, -0.0989,
         0.0458,  0.1487,  0.1613, -0.1923, -0.2011,  0.0220,  0.1711, -0.1723,
        -0.1282,  0.1162,  0.1062,  0.2638, -0.2141, -0.1521, -0.1472, -0.1864,
         0.2665,  0.1019,  0.1667,  0.0157, -0.0681,  0.2228, -0.0623, 

In [125]:
model.state_dict()['layer1.weight']/torch.sum(model.state_dict()['layer1.weight'])

tensor([[ 0.0072, -0.0133, -0.0046,  ...,  0.0002, -0.0011,  0.0033],
        [-0.0106,  0.0109, -0.0010,  ..., -0.0008,  0.0038, -0.0041],
        [ 0.0095, -0.0081,  0.0064,  ...,  0.0075,  0.0092,  0.0075],
        ...,
        [ 0.0066,  0.0098, -0.0030,  ...,  0.0018,  0.0064, -0.0092],
        [ 0.0043, -0.0067, -0.0051,  ...,  0.0022,  0.0028,  0.0063],
        [ 0.0013,  0.0052, -0.0016,  ..., -0.0030, -0.0103, -0.0048]])

In [45]:
a=server.tracker_temp

In [46]:
a

tensor([0., 0., 0., 1.])

In [47]:
layer_params_dict['layer1.weight']

(tensor([[ 0.1397, -0.2572, -0.0890,  ...,  0.0035, -0.0211,  0.0634],
         [-0.2041,  0.2103, -0.0196,  ..., -0.0148,  0.0740, -0.0785],
         [ 0.1840, -0.1573,  0.1232,  ...,  0.1452,  0.1784,  0.1457],
         ...,
         [ 0.1272,  0.1901, -0.0585,  ...,  0.0350,  0.1233, -0.1788],
         [ 0.0828, -0.1304, -0.0977,  ...,  0.0419,  0.0551,  0.1226],
         [ 0.0256,  0.1011, -0.0308,  ..., -0.0581, -0.1986, -0.0934]]),
 tensor([[ 0.0968, -0.0463, -0.0150,  ..., -0.0435, -0.0823,  0.1113],
         [ 0.0008,  0.0966,  0.0209,  ..., -0.0283, -0.0732, -0.0203],
         [ 0.1599, -0.0813,  0.1718,  ...,  0.0765,  0.0846,  0.0169],
         ...,
         [ 0.1794,  0.0361,  0.0829,  ...,  0.0166,  0.0558,  0.0058],
         [ 0.0542, -0.1225,  0.0591,  ..., -0.0765,  0.1391, -0.0342],
         [-0.0202,  0.0192,  0.0778,  ...,  0.0370, -0.1329, -0.0143]]),
 tensor([[ 0.1077, -0.1120, -0.0364,  ..., -0.0214, -0.0503,  0.0946],
         [-0.0565,  0.1306,  0.0082,  ..., -0

In [24]:
aggregated_parameters = {}
name='layer1.weight'

aggregated_parameters[name] = torch.sum(
    a
    / a.sum()
    * torch.stack(layer_params_dict[name], dim=-1).to(server.device),
    dim=-1,
)

In [25]:
torch.sum(a/ a.sum()* torch.stack(layer_params_dict[name], dim=-1).to(server.device),dim=-1)

tensor([[ 0.1097, -0.1156, -0.0376,  ..., -0.0243, -0.0548,  0.0958],
        [-0.0626,  0.1329,  0.0079,  ..., -0.0289, -0.0272, -0.0417],
        [ 0.1667, -0.1080,  0.1538,  ...,  0.0941,  0.1139,  0.0508],
        ...,
        [ 0.1596,  0.0830,  0.0363,  ...,  0.0198,  0.0742, -0.0490],
        [ 0.0621, -0.1171,  0.0094,  ..., -0.0394,  0.1077,  0.0181],
        [-0.0101,  0.0412,  0.0435,  ...,  0.0055, -0.1531, -0.0407]])

In [49]:
aggregated_parameters[name].shape

torch.Size([512, 14])

In [25]:
with open("D:/Download/pythonProject/HiWi/pFedLA_Folktable/src/pkl_files/hn/0.pkl", 'rb') as f:
    qw = pickle.load(f)
    
qw["mlp"][0].weight

Parameter containing:
tensor([[ 0.0273,  0.0562, -0.0342,  ...,  0.0298, -0.0126, -0.0052],
        [ 0.0328, -0.0955, -0.1000,  ..., -0.0489,  0.0381,  0.0211],
        [-0.0320,  0.0745, -0.0512,  ..., -0.0103,  0.0453, -0.0010],
        ...,
        [ 0.0384, -0.0118,  0.0080,  ..., -0.0856, -0.0647, -0.0319],
        [ 0.0429,  0.0723,  0.0675,  ..., -0.0407, -0.0915,  0.0258],
        [-0.0868, -0.0254, -0.0313,  ..., -0.0187, -0.0333,  0.0142]],
       requires_grad=True)

In [9]:
# D:\Download\pythonProject\HiWi\pFedLA_Folktable\temp\hn
with open("D:/Download/pythonProject/HiWi/pFedLA-master_original/temp/hn/8.pkl", 'rb') as f:
    a1 = pickle.load(f)
    
a1["mlp"][0].weight

Parameter containing:
tensor([[-0.0937, -0.0340, -0.0789,  ...,  0.0953, -0.0907,  0.0428],
        [-0.0973, -0.0327,  0.0957,  ..., -0.0736,  0.0715,  0.0400],
        [ 0.0272, -0.0741, -0.0024,  ...,  0.0134,  0.0945, -0.0741],
        ...,
        [-0.0915,  0.0650, -0.0139,  ..., -0.0919,  0.0284, -0.0107],
        [ 0.0486, -0.0921, -0.0964,  ...,  0.0980, -0.0358,  0.0721],
        [ 0.0323, -0.0272,  0.0640,  ..., -0.0886,  0.0630,  0.0817]],
       requires_grad=True)

In [32]:
# D:\Download\pythonProject\HiWi\pFedLA_Folktable\temp\hn
with open("D:/Download/pythonProject/HiWi/pFedLA-master_original/temp/hn/8.pkl", 'rb') as f:
    a2 = pickle.load(f)
    
a2["mlp"][0].weight[0]

tensor([-0.0937, -0.0340, -0.0789,  0.0736, -0.0949,  0.0642,  0.0790,  0.0037,
        -0.0856,  0.0770, -0.0534, -0.0941,  0.0115,  0.0501, -0.0512,  0.0796,
        -0.0407,  0.0587, -0.0320, -0.0691,  0.0732,  0.0862, -0.0808,  0.0117,
        -0.0041, -0.0942,  0.0254, -0.0815, -0.0757,  0.0142, -0.0595, -0.0525,
        -0.0617, -0.0892, -0.0879, -0.0020, -0.0862,  0.0740, -0.0417,  0.0953,
         0.0618,  0.0115, -0.0543,  0.0179, -0.0213, -0.0651,  0.0661,  0.0287,
        -0.0900, -0.0155,  0.0535, -0.0633, -0.0122,  0.0616,  0.0409, -0.0100,
        -0.0223, -0.0163,  0.0374,  0.0285, -0.0653,  0.0042, -0.0179, -0.0476,
         0.0977,  0.0163,  0.0297, -0.0257, -0.0303, -0.0456, -0.0347, -0.0898,
         0.0068, -0.0229, -0.0901, -0.0588,  0.0831, -0.0576, -0.0739,  0.0262,
         0.0600,  0.0905, -0.0072, -0.0443,  0.0981,  0.0350, -0.0329, -0.0846,
        -0.0869,  0.0158,  0.0663, -0.0791,  0.0413, -0.0424, -0.0297, -0.0697,
        -0.0594,  0.0953, -0.0907,  0.04

In [33]:
torch.equal(a1["mlp"][0].weight[0],a2["mlp"][0].weight[0])

True

In [2]:
import torch

# Create a 3x3x3 tensor
tensor = torch.tensor([
    [[1, 2, 3],
     [4, 5, 6],
     [7, 8, 9]],
    
    [[10, 11, 12],
     [13, 14, 15],
     [16, 17, 18]],
    
    [[19, 20, 21],
     [22, 23, 24],
     [25, 26, 27]]
])

# Calculate sum along the last dimension (dim=-1)
sum_along_last_dim = torch.sum(tensor, dim=-1)

print(sum_along_last_dim)


tensor([[ 6, 15, 24],
        [33, 42, 51],
        [60, 69, 78]])
